## Data output manipulation

In [114]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import glob
import re
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from sklearn.decomposition import PCA
from time import time
import math
from sklearn import cross_validation

In [117]:
# constants

data = '/kaggle/dev/data-science-bowl-2017-data/'
stage1 = '/kaggle/dev/data-science-bowl-2017-data/stage1/'
labels = '/kaggle/dev/data-science-bowl-2017-data/stage1_labels.csv'
stage1_processed = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed/'
stage1_features_resnet = '/kaggle/dev/data-science-bowl-2017-data/stage1_features_mx/'
stage1_submission = '/kaggle/dev/data-science-bowl-2017-data/stage1_sample_submission.csv'
naive_submission = '/kaggle/dev/jovan/data-science-bowl-2017/data-science-bowl-2017/submissions/naive_submission.csv'
stage1_processed_pca = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed_pca/'
stage1_features_inception = '/kaggle/dev/data-science-bowl-2017-data/CIFAR-10/cache/'
submissions = '/kaggle/dev/data-science-bowl-2017-data/submissions/'

cifar10_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
cifar_data = "/kaggle/dev/data-science-bowl-2017-data/CIFAR-10/"

In [255]:
ids = list()
for s in glob.glob(stage1_features_inception + "*"):
    id = os.path.basename(s)
    id = re.match(r'inception_cifar10_([a-f0-9].*).pkl' , id).group(1)
    ids.append(id)
ids = pd.DataFrame(ids,  columns=["id"])

df = pd.read_csv(labels)
df = pd.merge(df, ids, how='inner', on=['id'])

x = np.array([np.mean(np.load(stage1_features_inception + "inception_cifar10_" + s + ".pkl"), axis=0) for s in df['id'].tolist()])
y = df['cancer'].as_matrix()
trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                test_size=0.20)

In [256]:
df

,id,cancer
0,0015ceb851d7251b8f399e39779d1e7d,1
1,0030a160d58723ff36d73f41b170ec21,0
2,003f41c78e6acfa92430a057ac0b306e,0
3,006b96310a37b36cccb2ab48d10b49a3,1
4,008464bb8521d09a42985dd8add3d0d2,1
5,0092c13f9e00a3717fdc940641f00015,0
6,00986bebc45e12038ef0ce3e9962b51a,0
7,00cba091fa4ad62cc3200a657aeb957e,0
8,00edff4f51a893d80dae2d42a7f45ad1,1
9,0121c2845f2b7df060945b072b2515d7,0


In [132]:
num_classes = 2

transfer_values_test = val_x
transfer_values_train = trn_x
cls_test = val_y
cls_train = trn_y
labels_test = (np.arange(num_classes) == val_y[:, None])+0
labels_train = (np.arange(num_classes) == trn_y[:, None])+0


print("transfer_values_test : " + str(transfer_values_test.shape))
print("transfer_values_train : " + str(transfer_values_train.shape))
print("labels_test : " + str(labels_test.shape))
print("labels_train : " + str(labels_train.shape))
print("cls_test : " + str(cls_test.shape))
print("cls_train : " + str(cls_train.shape))

transfer_values_test : (280, 2048)
transfer_values_train : (1117, 2048)
labels_test : (280, 2)
labels_train : (1117, 2)
cls_test : (280,)
cls_train : (1117,)


In [131]:
labels_test.shape

(280,)

In [123]:
biggest_0 = 0
biggest_total = 0
for i in range(0, len(transfer_values_test)):
    x = transfer_values_test[i].shape[0] * transfer_values_test[i].shape[1]
    if x > biggest_total:
        biggest_total = x
        biggest_0 = transfer_values_test[i].shape[0]
        
print(biggest_0)
print(biggest_total)

428
876544


In [233]:
def test(temp, **labels):
    print(temp)
    if labels:
        labels = (labels['labels'])
        print(len(labels))
    print(len(labels))
    if temp=="ok":
        joel = 2
        print(joel)
    print(joel)
    

In [234]:
test("ok", labels=[0,1,2,3])

ok
4
4
2
2


In [244]:
temp = np.array([[ 0.71215874 , 0.28784126]])

In [245]:
temp.shape

(1, 2)

In [246]:
np.amax(temp)

0.71215874000000001